In [ ]:
# SPAI: AI-Generated Image Detector - Google Colab Setup

# Step 1: Clone the repository
!git clone https://github.com/mever-team/spai.git
%cd spai

fatal: destination path 'spai' already exists and is not an empty directory.
/content/spai


In [ ]:
!pip uninstall torch torchvision torchaudio -y
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

Found existing installation: torch 2.5.1+cu121
Uninstalling torch-2.5.1+cu121:
  Successfully uninstalled torch-2.5.1+cu121
Found existing installation: torchvision 0.20.1+cu121
Uninstalling torchvision-0.20.1+cu121:
  Successfully uninstalled torchvision-0.20.1+cu121
Found existing installation: torchaudio 2.5.1+cu121
Uninstalling torchaudio-2.5.1+cu121:
  Successfully uninstalled torchaudio-2.5.1+cu121
Looking in indexes: https://download.pytorch.org/whl/cu121
  Using cached https://download.pytorch.org/whl/cu121/torch-2.5.1%2Bcu121-cp311-cp311-linux_x86_64.whl (780.5 MB)
  Using cached https://download.pytorch.org/whl/cu121/torchvision-0.20.1%2Bcu121-cp311-cp311-linux_x86_64.whl (7.3 MB)
  Using cached https://download.pytorch.org/whl/cu121/torchaudio-2.5.1%2Bcu121-cp311-cp311-linux_x86_64.whl (3.4 MB)


In [ ]:
# Step 2: Install dependencies
!pip install -r requirements.txt
!pip install filetype

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-31w39eqe
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-31w39eqe
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done


In [ ]:
# Step 3: Create weights directory and download pretrained model
!mkdir -p weights
!gdown 1vvXmZqs6TVJdj8iF1oJ4L_fcgdQrp_YI -O weights/

Downloading...
From (original): https://drive.google.com/uc?id=1vvXmZqs6TVJdj8iF1oJ4L_fcgdQrp_YI
From (redirected): https://drive.google.com/uc?id=1vvXmZqs6TVJdj8iF1oJ4L_fcgdQrp_YI&confirm=t&uuid=9caa33ae-5a0b-4f65-af6a-60dae78bae51
To: /content/spai/weights/spai.pth
100% 935M/935M [00:21<00:00, 43.5MB/s]


In [ ]:
# Step 4: Create input and output directories
!mkdir -p input_images output_results

In [ ]:
# Step 5: Clear previous images and upload new ones
import os
import shutil
from google.colab import files

# Clear the input directory of any previous files
if os.path.exists('input_images'):
    shutil.rmtree('input_images')
os.makedirs('input_images', exist_ok=True)

print("Upload your images to test:")
uploaded = files.upload()

# Move uploaded files to input directory
for filename in uploaded.keys():
    src_path = filename
    dst_path = os.path.join('input_images', filename)
    shutil.move(src_path, dst_path)
    print(f"Moved {filename} to input_images/")

print(f"Total files in input_images: {len(os.listdir('input_images'))}")

Upload your images to test:


Saving hd_real_xray_2.jpg to hd_real_xray_2.jpg
Saving fake_xray3.png to fake_xray3.png
Saving fake_xray2.png to fake_xray2.png
Saving fake_xray.png to fake_xray.png
Saving real_xray.jpg to real_xray.jpg
Moved hd_real_xray_2.jpg to input_images/
Moved fake_xray3.png to input_images/
Moved fake_xray2.png to input_images/
Moved fake_xray.png to input_images/
Moved real_xray.jpg to input_images/
Total files in input_images: 5


In [ ]:
# Step 6: Clear GPU memory and run inference with memory optimization
import torch
import gc

# Clear GPU cache
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    gc.collect()

# Set environment variables for memory optimization
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

# Run inference with reduced batch size and feature extraction batch size
!python -m spai infer \
    --input input_images \
    --output output_results \
    --opt "MODEL.FEATURE_EXTRACTION_BATCH" "50" \
    --opt "DATA.BATCH_SIZE" "1"

2025-08-10 20:54:35.601290: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754859275.625070   13720 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754859275.631788   13720 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1754859275.648179   13720 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1754859275.648209   13720 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1754859275.648215   13720 computation_placer.cc:177] computation placer alr

In [ ]:
# Step 7: View results (Fixed CSV parsing)
import pandas as pd
import os

# Find the CSV result file
result_files = [f for f in os.listdir('output_results') if f.endswith('.csv')]
if result_files:
    # Read the CSV file
    results_df = pd.read_csv(f'output_results/{result_files[0]}')
    print("Raw CSV Data:")
    print("=" * 50)
    print(results_df)
    print(f"\nColumns: {list(results_df.columns)}")
    print(f"Shape: {results_df.shape}")

    # Try to find the score column (usually named 'spai' or similar)
    print("\nDetection Results:")
    print("=" * 50)

    for idx, row in results_df.iterrows():
        filename = row['image'] if 'image' in results_df.columns else row.iloc[0]

        # Look for score in different possible columns
        score = None
        if 'spai' in results_df.columns:
            score = row['spai']
        elif len(results_df.columns) > 3:  # Last column is usually the score
            score = row.iloc[-1]

        if score is not None and isinstance(score, (int, float)):
            if score > 0.1:
                prediction = "🤖 AI-Generated"
                confidence = f"{score:.3f}"
            else:
                prediction = "📷 Real Photo"
                confidence = f"{score:.3f}"

            print(f"{filename}")
            print(f"  → {prediction} (score: {confidence})")
            print()
        else:
            print(f"{filename}: Unable to parse score - {score}")
else:
    print("No result files found!")

# Also try to read the raw file content
print("\n" + "="*50)
print("Raw file content:")
if result_files:
    with open(f'output_results/{result_files[0]}', 'r') as f:
        content = f.read()
        print(content[:500])  # First 500 characters

Raw CSV Data:
                image split  class          spai
0      fake_xray2.png  test      1  3.821575e-02
1  hd_real_xray_2.jpg  test      1  1.268502e-06
2       fake_xray.png  test      1  5.532836e-01
3      fake_xray3.png  test      1  1.602456e-06
4       real_xray.jpg  test      1  1.961786e-14

Columns: ['image', 'split', 'class', 'spai']
Shape: (5, 4)

Detection Results:
fake_xray2.png
  → 📷 Real Photo (score: 0.038)

hd_real_xray_2.jpg
  → 📷 Real Photo (score: 0.000)

fake_xray.png
  → 🤖 AI-Generated (score: 0.553)

fake_xray3.png
  → 📷 Real Photo (score: 0.000)

real_xray.jpg
  → 📷 Real Photo (score: 0.000)


Raw file content:
image,split,class,spai
fake_xray2.png,test,1,0.0382157526910305
hd_real_xray_2.jpg,test,1,1.2685015917668352e-06
fake_xray.png,test,1,0.5532836318016052
fake_xray3.png,test,1,1.6024564502004068e-06
real_xray.jpg,test,1,1.9617860734879623e-14



In [ ]:
# Step 8: Download results (optional)
from google.colab import files
if result_files:
    files.download(f'output_results/{result_files[0]}')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>